## Day 14. Task 05
## Пайплайны и ООП
### 0. Импорты

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from joblib import dump, load
pd.set_option('display.max_colwidth', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. Пайплайн препроцессинга
 Создайте три кастомных трансформера согласно указаниям ниже. Первые два трансформера из списка будут использованы внутри Pipeline.
1. Класс FeatureExtractor():
    - принимает на вход датафрейм с полями uid, labname, numTrials, timestamp из файла datasets/checker_submits.csv
    - вытаскивает hour из timestamp
    - вытаскивает weekday из timestamp (сохраняет в виде цифр)
    - удаляет столбец timestamp
    - возвращает новый датафрейм
2. Класс MyOneHotEncoder():
    - принимает на вход датафрейм из предыдущего трансформера и название столбца с целевой переменной
    - обнаруживает все категориальные признаки и трансормирует их при помощи OneHotEncoder(). Если целевая переменная является тоже категориальной, то трансформация на нее не распространяется
    - удаляет изначальные категориальные столбцы из датафрейма
    - возвращает датафрейм с признаками и series со столбцом целевой переменной
3. Класс TrainValidationTest():
    - принимает на вход X и y
    - возвращает X_train, X_valid, X_test, y_train, y_valid, y_test (test_size=0.2, random_state=21, stratified).

In [3]:
class FeatureExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        X['hour'] = X['timestamp'].dt.hour
        X['dayofweek'] = X['timestamp'].dt.weekday
        X = X.drop(columns = ['timestamp'])
        return X

In [4]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, target):
        self.target = target
    
    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        ohe = OneHotEncoder(sparse=False)
        X_cat = X.drop(columns=[self.target]).select_dtypes(include = ['object'])
        X_num = X.drop(columns=[self.target]).select_dtypes(exclude = ['object'])
        X_dum = pd.DataFrame(data = ohe.fit_transform(X_cat), columns=ohe.get_feature_names())
        X_trans = X_dum.join(X_num)
        return X_trans, X[self.target]

In [5]:
class TrainValidationTest():

    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def train_valid_test_split(self):
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=21)
        X_train, X_valid, y_train, y_validt = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=21)
        return X_train, X_valid, X_test,  y_train, y_validt, y_test

### 2. Пайплайн выбора модели
 Класс `ModelSelection()`
  - Принимает на вход список экземпляров `GridSearchCV` и словарь, в котором ключами являются индексы из этого списка, а значениями – названия моделей. Ниже пример в обратном порядке (для лучшего понимания, строки кода не будут работать при прогоне). Такой порядок позволяет посмотреть на это от общего к частному:
```ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```
  - метод `choose()` принимает на вход `X_train`, `y_train`, `X_valid`, `y_valid` и возвращает название лучшего классификатора среди всех моделей на валидационной выборке
  - метод `best_results()` возвращает датафрейм со столбцами `model`, `params`, `valid_score`, где строки – это модели, являющиеся лучшими в своем классе моделей (пример ниже, ваши значения будут другими)
  ```
model    params    valid_score
0    SVM    {'C': 10, 'class_weight': None, 'gamma': 'auto...    0.772727
1    Decision Tree    {'class_weight': 'balanced', 'criterion': 'gin...    0.801484
2    Random Forest    {'class_weight': None, 'criterion': 'entropy',...    0.855288
```
  - когда вы итерируетесь по параметрам класса моделей, выводите на экран название этого класса, а также показывайте прогресс, используя `tqdm.notebook`, в конце цикла этого класса моделей выведете на экран лучшую модель этого класса (ниже пример, значения могут отличаться)
```
Estimator: SVM
100%
72/72 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
196/196 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
784/784 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [6]:
class ModelSelection():
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict

    def choose(self, X_train, y_train, X_valid, y_valid):
        self.model = []
        self.params = []
        self.valid_score = []
        best_name = None
        best_score = 0
        for i in tqdm(range(len(self.grids))):
            print(f'Estimator: {grid_dict[i]}')
            self.grids[i].fit(X_train, y_train)
            print(f'Best params: {self.grids[i].best_params_}')
            print(f'Best training accuracy: {self.grids[i].best_score_}')
            self.grids[i].best_estimator_.fit(X_train, y_train)
            valid_score = accuracy_score(y_valid, self.grids[i].best_estimator_.predict(X_valid))
            print(f'Validation set accuracy score for best params: {valid_score}\n')
            self.model.append(grid_dict[i])
            self.params.append(self.grids[i].best_params_)
            self.valid_score.append(valid_score)
            if valid_score > best_score:
                best_name = grid_dict[i]
                best_score = valid_score
        print(f'Classifier with best validation set accuracy: {best_name}')

    def best_results(self):
        return pd.DataFrame(data = {'model': self.model, 'params': self.params, 'valid_score': self.valid_score})

### 3. Финализация
 Класс `Finalize()`
   - принимает на вход estimator (класс, который оценивает качество модели)
   - у класса должен быть метод `save_model()`, который сохраняет финальную модель с заданным путем и именем и сообщает, что модель успешно сохранена, а также метод `final_score()`, который принимает на вход `X_train`, `y_train`, `X_test`, `y_test` и возвращает accuracy модели, как в примере ниже:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 

In [7]:
class Finalize():
    def __init__(self, estimator):
        self.estimator = estimator

    def final_score (self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        return f'Accuracy of the final model is {accuracy_score(y_test, self.estimator.predict(X_test))}'

    def save_model(self, filename):
        dump(self.estimator, open(filename, 'wb'))

### 4. Основная программа
 
1. Загрузите данные из файла.
2. Создайте пайплайн препроцессинга, который состоит из двух написанных вами трансформеров: `FeatureExtractor()` и `MyOneHotEncoder()`.
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Примените этот пайплайн и его метод `fit_transform()` по отношению к изначальному датасету.
```
data = preprocessing.fit_transform(df)
```
4. Получите `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test`, используя `TrainValidationTest()` и результат предыдущего пайплайна.
5. Создайте объект класса `ModelSelection()`, воспользуйтесь методом `choose()`, применив его к тем моделям и тем параметрам моделей, которые вы хотите. Получите датафрейм с лучшими результатами.
6. Создайте объект класса `Finalize()` с вашей лучшей моделью. Воспользуйтесь методом `final_score()` и сохраните модель в формате `name_of_the_model_{accuracy on test dataset}.sav.`  
Это всё. Поздравляем! Это серьезный результат. Вы смогли автоматизировать свою работу.

In [8]:
df = pd.read_csv('../content/drive/MyDrive/School21/day14/data/checker_submits.csv', parse_dates=['timestamp'])
pipe = make_pipeline(FeatureExtractor(), MyOneHotEncoder(target='dayofweek'))
X, y = pipe.fit_transform(df)
X_train, X_valid, X_test,  y_train, y_valid, y_test = TrainValidationTest(X, y).train_valid_test_split()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
svm = SVC()
svm_params =  {'kernel' : ['linear', 'rbf', 'sigmoid'],
              'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None],
              'random_state': [21],
              'probability': [True]}
gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)

dt = DecisionTreeClassifier()
dt_params  = {'max_depth': range(1,50),
              'class_weight': ['balanced', None],
              'criterion': ['entropy', 'gini'],
              'random_state': [21]}
gs_dt = GridSearchCV(estimator=dt, param_grid=dt_params, scoring='accuracy', cv=2, n_jobs=-1)

rf = RandomForestClassifier()
rf_params  = {'max_depth': range(1,50),
              'class_weight': ['balanced', None],
              'criterion': ['entropy', 'gini'],
              'n_estimators': [5, 10, 50, 100],
              'random_state': [21]}
gs_rf = GridSearchCV(estimator=rf, param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)

grids = [gs_svm, gs_dt, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}


In [10]:
ms = ModelSelection(grids=grids, grid_dict=grid_dict)
ms.choose(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid)
best_results = ms.best_results()
best_results

  0%|          | 0/3 [00:00<?, ?it/s]

Estimator: SVM
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.7727272727272727
Validation set accuracy score for best params: 0.8777777777777778

Estimator: Decision Tree
Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 17, 'random_state': 21}
Best training accuracy: 0.7996289424860854
Validation set accuracy score for best params: 0.8703703703703703

Estimator: Random Forest
Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 27, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.8571428571428572
Validation set accuracy score for best params: 0.8925925925925926

Classifier with best validation set accuracy: Random Forest


,model,params,valid_score
0,SVM,"{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}",0.877778
1,Decision Tree,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 17, 'random_state': 21}",0.870370
2,Random Forest,"{'class_weight': None, 'criterion': 'gini', 'max_depth': 27, 'n_estimators': 50, 'random_state': 21}",0.892593


In [11]:
best_params = best_results.sort_values('valid_score', ascending=False).reset_index(drop=True).loc[0, 'params']
best_params

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 27,
 'n_estimators': 50,
 'random_state': 21}

In [12]:
best_model = RandomForestClassifier(**best_params)
best_model

RandomForestClassifier(max_depth=27, n_estimators=50, random_state=21)

In [13]:
final = Finalize(best_model)
final.final_score(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

'Accuracy of the final model is 0.9171597633136095'

In [14]:
final.save_model('RandomForestClassifier_0.9171597633136095.sav')

In [15]:
best_model_1 = load(open('RandomForestClassifier_0.9171597633136095.sav', 'rb'))

In [16]:
best_model_1

RandomForestClassifier(max_depth=27, n_estimators=50, random_state=21)